In [ ]:
import geopandas as gpd
import joblib
import numpy as np
import odc.geo  # noqa
from matplotlib import colors
from shapely import geometry

from utils import load_data, predict_xr

In [ ]:
%reload_ext autoreload
%autoreload 2

## Find and load S2 data

Load data and set up your array to use for prediction

In [ ]:
# Configure some things up front
chunks = dict(x=2048, y=2048)
datetime = "2023"

bbox = [177.35, -17.85, 177.50, -17.70]
bbox_geometry = geometry.box(*bbox)

gdf = gpd.GeoDataFrame({'geometry': [bbox_geometry]}, crs='EPSG:4326')
gdf.explore()

In [ ]:
merged = load_data(bbox, datetime=datetime, chunks=chunks, resolution=10)
merged

In [ ]:
loaded_model = joblib.load("logistic_model.dump")

In [ ]:
loaded_data = merged.compute()

## Run ML classifier

In [ ]:
# This runs the actual prediction
predicted = predict_xr(loaded_model, loaded_data, proba=True)

# Convert to int
cleaned_predictions = predicted.copy(deep=True)
cleaned_predictions.predictions.data = predicted.predictions.data.astype(np.int8)
cleaned_predictions.probabilities.data = predicted.probabilities.data.astype(np.float32)

cleaned_predictions = cleaned_predictions.rename({"predictions": "pred", "probabilities": "prob"})

In [ ]:
cleaned_predictions.prob.odc.explore(cmap="Reds")

In [ ]:
cleaned_predictions.pred.odc.explore(cmap="Reds")

In [ ]:
# Write GeoTIFF
cleaned_predictions.lulc.odc.write_cog("lulc_nadi_2023_3.tif", overwrite=True)